In [81]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import countDistinct, col, monotonically_increasing_id, count

In [89]:
spark = SparkSession.builder.master("local[4]") \
                    .appName('recommendation_system') \
                    .config("spark.driver.memory", "15g") \
                    .getOrCreate()

In [3]:
sc = spark.sparkContext

## Data Download

In [92]:
!mkdir -p data
!kaggle datasets download -d grouplens/movielens-20m-dataset -p data/ --unzip

 99%|███████████████████████████████████████▍| 193M/195M [00:06<00:00, 54.0MB/s]
100%|████████████████████████████████████████| 195M/195M [00:06<00:00, 32.3MB/s]


In [108]:
df = spark.read.options(
            header=True,
            inferSchema=True
        ) \
        .csv('data/rating.csv')

In [109]:
df.count()

20000263

In [110]:
df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [111]:
df.show(3)

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|      2|   3.5|2005-04-02 23:53:47|
|     1|     29|   3.5|2005-04-02 23:31:16|
|     1|     32|   3.5|2005-04-02 23:33:39|
+------+-------+------+-------------------+
only showing top 3 rows



## Fixing User IDs and Movie IDs for easy indexing

In [112]:
print(f"The unique number of users is {df.select(countDistinct('userId')).collect()[0][0]}\n"
      f"User ID starts from {df.agg({'userId': 'min'}).collect()[0][0]}\n"
      f"User ID ends at {df.agg({'userId': 'max'}).collect()[0][0]}"
)

The unique number of users is 138493
User ID starts from 1
User ID ends at 138493


In [113]:
df = df.withColumn("userId",col("userId")-1)

In [114]:
print(f"The unique number of users is {df.select(countDistinct('userId')).collect()[0][0]}\n"
      f"User ID starts from {df.agg({'userId': 'min'}).collect()[0][0]}\n"
      f"User ID ends at {df.agg({'userId': 'max'}).collect()[0][0]}"
)

The unique number of users is 138493
User ID starts from 0
User ID ends at 138492


In [115]:
print(f"The unique number of movies is {df.select(countDistinct('movieId')).collect()[0][0]}\n"
      f"Movie ID starts from {df.agg({'movieId': 'min'}).collect()[0][0]}\n"
      f"Movie ID ends at {df.agg({'movieId': 'max'}).collect()[0][0]}"
)

The unique number of movies is 26744
Movie ID starts from 1
Movie ID ends at 131262


In [116]:
movieId_df = df.select('movieId').distinct().sort('movieId')
movieId_df = movieId_df.withColumn("movieIdx", monotonically_increasing_id())
movieId_df.show()

+-------+--------+
|movieId|movieIdx|
+-------+--------+
|      1|       0|
|      2|       1|
|      3|       2|
|      4|       3|
|      5|       4|
|      6|       5|
|      7|       6|
|      8|       7|
|      9|       8|
|     10|       9|
|     11|      10|
|     12|      11|
|     13|      12|
|     14|      13|
|     15|      14|
|     16|      15|
|     17|      16|
|     18|      17|
|     19|      18|
|     20|      19|
+-------+--------+
only showing top 20 rows



In [117]:
movieId_df.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- movieIdx: long (nullable = false)



In [118]:
df.show()

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     0|      2|   3.5|2005-04-02 23:53:47|
|     0|     29|   3.5|2005-04-02 23:31:16|
|     0|     32|   3.5|2005-04-02 23:33:39|
|     0|     47|   3.5|2005-04-02 23:32:07|
|     0|     50|   3.5|2005-04-02 23:29:40|
|     0|    112|   3.5|2004-09-10 03:09:00|
|     0|    151|   4.0|2004-09-10 03:08:54|
|     0|    223|   4.0|2005-04-02 23:46:13|
|     0|    253|   4.0|2005-04-02 23:35:40|
|     0|    260|   4.0|2005-04-02 23:33:46|
|     0|    293|   4.0|2005-04-02 23:31:43|
|     0|    296|   4.0|2005-04-02 23:32:47|
|     0|    318|   4.0|2005-04-02 23:33:18|
|     0|    337|   3.5|2004-09-10 03:08:29|
|     0|    367|   3.5|2005-04-02 23:53:00|
|     0|    541|   4.0|2005-04-02 23:30:03|
|     0|    589|   3.5|2005-04-02 23:45:57|
|     0|    593|   3.5|2005-04-02 23:31:01|
|     0|    653|   3.0|2004-09-10 03:08:11|
|     0|    919|   3.5|2004-09-1

In [119]:
ratings = df.join(
    movieId_df,
    df.movieId == movieId_df.movieId,
    "inner"
) \
    .drop('movieId') \
    .drop('timestamp')

In [120]:
ratings.show()

+------+------+--------+
|userId|rating|movieIdx|
+------+------+--------+
|     0|   3.5|    3903|
|     1|   4.0|    1528|
|     1|   3.0|    3825|
|     2|   4.0|    2281|
|     6|   3.0|    1528|
|     6|   2.0|    3088|
|     8|   2.0|    4424|
|    10|   5.0|    1528|
|    10|   5.0|    1539|
|    13|   5.0|     467|
|    13|   3.5|    1528|
|    13|   4.5|    3088|
|    13|   4.5|   10371|
|    13|   4.0|   10864|
|    15|   4.0|    1528|
|    16|   4.0|    1528|
|    17|   2.5|    1528|
|    20|   3.0|    2780|
|    21|   3.0|    1528|
|    22|   5.0|    1528|
+------+------+--------+
only showing top 20 rows



In [121]:
print(f"The unique number of users is {ratings.select(countDistinct('userId')).collect()[0][0]}\n"
      f"User ID starts from {ratings.agg({'userId': 'min'}).collect()[0][0]}\n"
      f"User ID ends at {ratings.agg({'userId': 'max'}).collect()[0][0]}"
)

The unique number of users is 138493
User ID starts from 0
User ID ends at 138492


In [122]:
print(f"The unique number of movies is {ratings.select(countDistinct('movieIdx')).collect()[0][0]}\n"
      f"Movie ID starts from {ratings.agg({'movieIdx': 'min'}).collect()[0][0]}\n"
      f"Movie ID ends at {ratings.agg({'movieIdx': 'max'}).collect()[0][0]}"
)

The unique number of movies is 26744
Movie ID starts from 0
Movie ID ends at 26743


In [123]:
ratings.write.options(header='True', delimiter=',').csv('./data/ratings_edited.csv')

In [124]:
spark.stop()